# Expected Goals (xG)

Expected goals measure the quality of a shot. This is achieved by calculating the likelihood that it will be scored from a particular position on the pitch during a particular phase of play. Sophisticated models take in a lot of factors including type of attack, type of shot, position on the pitch from where shot was taken etc. xG is measured on a scale between zero and one, where zero represents a chance that is impossible to score and one represents a chance that a player would be expected to score every single time.

## xG with Analysis

In [ ]:
#importing all required modules
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import scipy as sp
from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
#using csv while for pandas dataframe
filename = 'events.csv'
events = pd.read_csv('input/events.csv')
shots = events[(events.event_type==1)]


In [ ]:
events.head()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0


# 1. xG Model

## 1.1 One-hot encoding

In [ ]:
#Using one hot encoding for goal and shot details
shots_prediction = shots.iloc[:,-6:]
dummies = pd.get_dummies(shots_prediction, columns=['location', 'bodypart','assist_method', 'situation'])
dummies.columns = ['is_goal', 'fast_break', 'loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right', 'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range', 'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 'left_foot', 'header', 'no_assist', 'assist_pass', 'assist_cross', 'assist_header', 'assist_through_ball', 'open_play', 'set_piece', 'corner', 'free_kick']
dummies.head()

,is_goal,fast_break,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,...,header,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
13,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [ ]:
X = dummies.iloc[:,1:]
y = dummies.iloc[:,0]
print(X.shape)
print(y.shape)

(229135, 28)
(229135,)


## 1.2 Test-Train split


In [ ]:
#Training and tesint split with 35% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

## 2.1 XGB Classifier

In [ ]:
classifier = XGBClassifier(objective='binary:logistic', max_depth=8, n_estimators=100)
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
#Checking accuracy/ROC scores
accuracy = classifier.score(X_test, y_test)
y_pred = classifier.predict_proba(X_test)
predict = classifier.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
auc_roc = roc_auc_score(y_test, y_pred[:, 1])
print(f'The training set has {y_total} examples of which {y_positive} are goals')
print(f'The model accuracy is {accuracy*100:.2f}%')
print(f'AUC-ROC is {auc_roc:.4f}')

The training set has 148937 examples of which 15937 are goals
The model accuracy is 91.09%
AUC-ROC is 0.8192


In [ ]:
auc_pr_baseline = y_positive / y_total
print(f'Baseline performance(random) for AUC-PR is {auc_pr_baseline:.2f}.')

auc_pr = average_precision_score(y_test, y_pred[:, 1])
print(f'AUC-PR = {auc_pr:.4f}.')
cohen_kappa = cohen_kappa_score(y_test,predict)
print(f'Cohen Kappa = {cohen_kappa:.4f}.')

Baseline performance(random) for AUC-PR is 0.11.
AUC-PR = 0.4739.
Cohen Kappa = 0.3519.


In [ ]:
#Confusion Matrix and checking precision, recall and f1 score
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict))
print('Report:')
print(classification_report(y_test,predict))

Confusion Matrix:
[[70773   921]
 [ 6225  2279]]
Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     71694
           1       0.71      0.27      0.39      8504

    accuracy                           0.91     80198
   macro avg       0.82      0.63      0.67     80198
weighted avg       0.90      0.91      0.89     80198



In [ ]:
predictions = X_test.copy()
predictions['true_goals'] = y_test
predictions['expected_goals'] = y_pred[:,1]
predictions['difference'] = predictions['expected_goals'] - predictions['true_goals']
predictions = predictions.iloc[:,28:31]
predictions.head()

,true_goals,expected_goals,difference
851561,0,0.179917,0.179917
744702,1,0.164767,-0.835233
661032,0,0.020609,0.020609
195074,0,0.179917,0.179917
742812,0,0.028105,0.028105


## 2.2 Logistic Regression

In [ ]:
#Same procedure implemented for linear regression
logistic_regression = LogisticRegression(class_weight='balanced')
logistic_regression.fit(X_train, y_train)

C:\Users\sukum\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
logistic_regression.score(X_train, y_train)
accuracy = logistic_regression.score(X_test, y_test)
y_pred = logistic_regression.predict_proba(X_test)
accuracy_logreg = logistic_regression.score(X_test, y_test)
y_pred_logreg = logistic_regression.predict_proba(X_test)
predict_logreg = logistic_regression.predict(X_test)
y_total_logreg = y_train.count()
y_positive_logreg = y_train.sum()
auc_roc_logreg = roc_auc_score(y_test, y_pred_logreg[:, 1])

print(f'The training set contains {y_total_logreg} examples of which {y_positive_logreg} are goals.')
print(f'Accuracy = {accuracy_logreg*100:.2f}%')
print(f'AUC-ROC = {auc_roc_logreg:.4f}.')

auc_pr_baseline = y_positive / y_total
print(f'Baseline performance for AUC-PR is {auc_pr_baseline:.4f}.')
auc_pr_logreg = average_precision_score(y_test, y_pred_logreg[:, 1])
print(f'AUC-PR = {auc_pr_logreg:.4f}.')
cohen_kappa_logreg = cohen_kappa_score(y_test,predict_logreg)
print(f'Cohen Kappa = {cohen_kappa_logreg:.4f}.')

The training set contains 148937 examples of which 15937 are goals.
Accuracy = 72.92%
AUC-ROC = 0.8185.
Baseline performance for AUC-PR is 0.1070.
AUC-PR = 0.4708.
Cohen Kappa = 0.2540.


In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict_logreg))
print('Report:')
print(classification_report(y_test,predict_logreg))

Confusion Matrix:
[[70820   874]
 [ 6265  2239]]
Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     71694
           1       0.72      0.26      0.39      8504

    accuracy                           0.91     80198
   macro avg       0.82      0.63      0.67     80198
weighted avg       0.90      0.91      0.89     80198



## 2.3 Neural Network (MLP)

In [ ]:
#Same procedure with Neural Networks 
mlp = MLPClassifier(random_state=0, hidden_layer_sizes=(28,28,28,28), max_iter=2000, activation='relu')
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(28, 28, 28, 28), learning_rate='constant',
              learning_rate_init=0.001, max_iter=2000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=0, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [ ]:
mlp.score(X_train, y_train)
mlp.score(X_test, y_test)
accuracy = mlp.score(X_test, y_test)
print(f'Accuracy = {accuracy*100:.2f}%.')
y_pred = mlp.predict_proba(X_test)
predict = mlp.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
print(f'The training set contains {y_total} examples of which {y_positive} are goals.')
auc_roc = roc_auc_score(y_test, y_pred[:,1])
print(f'AUC-ROC = {auc_roc:.4f}./n')
auc_pr_baseline = y_positive / y_total
print(f'Baseline performance for AUC-PR is {auc_pr_baseline:.4f}.')
auc_pr = average_precision_score(y_test, y_pred[:,1])
print(f'AUC-PR = {auc_pr:.4f}.')
cohen_kappa = cohen_kappa_score(y_test,predict)
print(f'Cohen Kappa = {cohen_kappa:.4f}.')
MSE = sklearn.metrics.mean_squared_error(y_test, y_pred[:,1])
print(f'Mean Squared Error (MSE) = {MSE:.4f}.')

Accuracy = 91.10%.
The training set contains 148937 examples of which 15937 are goals.
AUC-ROC = 0.8189./n
Baseline performance for AUC-PR is 0.1070.
AUC-PR = 0.4741.
Cohen Kappa = 0.3533.
Mean Squared Error (MSE) = 0.0726.


In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict))
print('Report:')
print(classification_report(y_test,predict))

Confusion Matrix:
[[70771   923]
 [ 6214  2290]]
Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     71694
           1       0.71      0.27      0.39      8504

    accuracy                           0.91     80198
   macro avg       0.82      0.63      0.67     80198
weighted avg       0.90      0.91      0.89     80198



In [ ]:
predictions = X_test.copy()
predictions['true_goals'] = y_test
predictions['expected_goals'] = y_pred[:,1]
predictions['difference'] = predictions['expected_goals'] - predictions['true_goals']
predictions = predictions.iloc[:,28:31]

# 3. Player Analysis

In [ ]:

ypred2 = mlp.predict_proba(X_train)
predictions_train = X_train.copy()
predictions_train['true_goals'] = y_train
predictions_train['expected_goals'] = ypred2[:,1]
predictions_train['difference'] = predictions_train['expected_goals'] - predictions_train['true_goals']
predictions_train = predictions_train.iloc[:,28:31]
all_predictions = pd.concat([predictions, predictions_train], axis=0)
events2 = pd.concat([events, all_predictions], axis=1)
shots2 = events2[events2.event_type==1]

## 3.1 Correlating goals scored and expected goals

In [ ]:
#Checking correlation between goals and expected goals
xG_players = shots2[['player', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player').sum()
xG_players.columns = ['n_shots', 'goals_scored', 'expected_goals', 'difference']
xG_players[['goals_scored', 'expected_goals']].corr()


,goals_scored,expected_goals
goals_scored,1.000000,0.976885
expected_goals,0.976885,1.000000


## 3.2 Players with best finishers

In [ ]:
#Player analysis 
xG_players.sort_values(['difference', 'goals_scored'])

,n_shots,goals_scored,expected_goals,difference
player,,,,
lionel messi,914,205.0,147.461680,-57.538321
luis suarez,433,96.0,65.747676,-30.252324
zlatan ibrahimovic,774,153.0,123.051116,-29.948884
gonzalo higuain,552,118.0,88.149569,-29.850431
cristiano ronaldo,1190,198.0,170.513715,-27.486285
robert lewandowski,633,124.0,98.861785,-25.138215
alexandre lacazette,391,88.0,64.810171,-23.189829
alexis sanchez,445,80.0,59.443467,-20.556533
antoine griezmann,493,80.0,60.042045,-19.957955


## 3.3 Players having high "expected goals"?

In [ ]:
#High expected goal players
xG_players.sort_values(['expected_goals'], ascending=False)

,n_shots,goals_scored,expected_goals,difference
player,,,,
cristiano ronaldo,1190,198.0,172.517022,-25.482978
lionel messi,914,205.0,147.785301,-57.214699
zlatan ibrahimovic,774,153.0,124.539731,-28.460269
edinson cavani,623,121.0,118.256201,-2.743799
robert lewandowski,633,124.0,99.958186,-24.041814
pierreemerick aubameyang,580,100.0,91.229665,-8.770335
gonzalo higuain,552,118.0,89.582126,-28.417874
antonio di natale,515,76.0,79.711294,3.711294
diego costa,410,93.0,75.104347,-17.895653


## 3.4 Players bad at picking their shots

In [ ]:
#Players with the lowest xG per shot ratio
xG_players['xG_per_shot_ratio'] = xG_players['expected_goals'] / xG_players['n_shots']
xG_players[xG_players.n_shots>100].sort_values(['xG_per_shot_ratio', 'goals_scored'])

,n_shots,goals_scored,expected_goals,difference,xG_per_shot_ratio
player,,,,,
tom huddlestone,108,2.0,3.801665,1.801665,0.035201
gokhan inler,185,9.0,7.333342,-1.666658,0.039640
florent balmont,168,5.0,7.122417,2.122417,0.042395
ivan radovanovic,161,2.0,7.569230,5.569230,0.047014
saphir taider,102,4.0,4.932113,0.932113,0.048354
ruben rochina,179,12.0,8.727522,-3.272478,0.048757
daniel baier,177,5.0,8.670070,3.670070,0.048983
michael pereira,101,3.0,4.961315,1.961315,0.049122
marco estrada,106,6.0,5.324807,-0.675193,0.050234


## 3.5 Best headers

In [ ]:
#Best headers of the ball
headers = events2[(events2.event_type==1) & (events2.bodypart==3)]
headers_players = headers[['player', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player').sum()
headers_players.columns = ['n_headers', 'goals_scored', 'expected_goals', 'difference']
headers_players.sort_values(['difference'])


,n_headers,goals_scored,expected_goals,difference
player,,,,
cristiano ronaldo,159,36.0,19.978661,-16.021339
mario mandzukic,124,28.0,14.458362,-13.541638
fernando llorente,141,25.0,15.727760,-9.272240
sloan privat,86,17.0,9.956783,-7.043217
pierreemerick aubameyang,59,14.0,7.339879,-6.660121
anthony modeste,104,17.0,10.829834,-6.170166
zlatan ibrahimovic,91,17.0,10.911163,-6.088837
michail antonio,32,12.0,5.916832,-6.083168
gareth bale,73,14.0,7.938579,-6.061421


## 3.6 left-footed goals

In [ ]:
#Best left footed shooters
left_foot = events2[(events2.event_type==1) & (events2.bodypart==2)]
left_foot_players = left_foot[['player', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player').sum()
left_foot_players.columns = ['n_left_foot_shots', 'goals_scored', 'expected_goals', 'difference']
left_foot_players.sort_values(['difference'])


,n_left_foot_shots,goals_scored,expected_goals,difference
player,,,,
lionel messi,752,167.0,122.223207,-44.776793
antoine griezmann,345,58.0,42.000227,-15.999773
iago falque,132,23.0,13.406378,-9.593622
arjen robben,296,42.0,32.459691,-9.540309
franck ribery,57,16.0,6.638735,-9.361265
mohamed salah,129,23.0,13.796967,-9.203033
iago aspas,180,34.0,25.352970,-8.647030
gareth bale,195,30.0,22.198496,-7.801504
mauro icardi,47,17.0,9.426798,-7.573202


## 3.7 Right-footed goals

In [ ]:
#Best right footed shooters
right_foot = events2[(events2.event_type==1) & (events2.bodypart==1)]
right_foot_players = right_foot[['player', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player').sum()
right_foot_players.columns = ['n_right_foot_shots', 'goals_scored', 'expected_goals', 'difference']
right_foot_players.sort_values(['difference'])

,n_right_foot_shots,goals_scored,expected_goals,difference
player,,,,
luis suarez,289,69.0,45.599689,-23.400311
gonzalo higuain,362,86.0,63.913154,-22.086846
alexandre lacazette,270,70.0,50.868442,-19.131558
robert lewandowski,378,84.0,68.662980,-15.337020
zlatan ibrahimovic,554,111.0,96.102880,-14.897120
alexis sanchez,352,62.0,47.829188,-14.170812
eden hazard,248,50.0,37.395238,-12.604762
karim benzema,270,54.0,41.413428,-12.586572
carlos tevez,181,33.0,21.269521,-11.730479


## 3.8 Best outside-the-box shooters

In [ ]:
#Best outside of box shooters
outside_box = shots2[(shots2.location==15)]
outbox_players = outside_box[['player', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player').sum()
outbox_players.columns = ['n_outside_box_shots', 'goals_scored', 'expected_goals', 'difference']
outbox_players.sort_values(['difference'])


,n_outside_box_shots,goals_scored,expected_goals,difference
player,,,,
lionel messi,304,16.0,6.958267,-9.041733
paul pogba,226,14.0,6.868845,-7.131155
gonzalo higuain,127,11.0,4.513068,-6.486932
james rodriguez,105,9.0,2.635160,-6.364840
yaya toure,128,10.0,3.964334,-6.035666
zlatan ibrahimovic,261,14.0,8.150821,-5.849179
alexandre lacazette,100,9.0,3.391386,-5.608614
blerim dzemaili,140,10.0,4.464851,-5.535149
sergio aguero,90,9.0,3.575988,-5.424012


## 3.9 Players with best passes

In [ ]:
#Passing metrics - best passers
passes_and_throughballs = pd.concat([shots2[shots2.assist_method==1], shots2[shots2.assist_method==4]])
assisting_players = passes_and_throughballs[['player2', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player2').sum()
assisting_players['xGoals_per_pass'] = assisting_players['expected_goals'] / assisting_players['event_type']
assisting_players.columns = ['n_passes', 'goals_scored_from_passes', 'xGoals_from_passes', 'difference', 'xGoals_per_pass']

assisting_players[assisting_players.n_passes > 100].sort_values(['xGoals_per_pass'], ascending=False)

,n_passes,goals_scored_from_passes,xGoals_from_passes,difference,xGoals_per_pass
player2,,,,,
luis suarez,185,40.0,31.000877,-8.999123,0.167572
gareth bale,109,21.0,17.690061,-3.309939,0.162294
angel di maria,211,43.0,32.743670,-10.256330,0.155183
lionel messi,350,68.0,52.762740,-15.237260,0.150751
neymar,179,25.0,26.439148,1.439148,0.147705
karim bellarabi,109,18.0,16.092844,-1.907156,0.147641
henrikh mkhitaryan,148,21.0,21.404461,0.404461,0.144625
marco verratti,113,21.0,16.328394,-4.671606,0.144499
pierreemerick aubameyang,130,21.0,18.465793,-2.534207,0.142045


## 3.10 Players with the best crosses

In [ ]:
#Best crossers
crosses = shots2[shots2.assist_method==2]
crosses_players = shots2[['player2', 'event_type', 'true_goals', 'expected_goals', 'difference']].groupby('player2').sum()
crosses_players.columns = ['n_crosses', 'goals_scored_from_crosses', 'xGoals_from_crosses', 'difference']
crosses_players['xGoals_per_cross'] = crosses_players['xGoals_from_crosses'] / crosses_players['n_crosses']
crosses_players.columns = ['n_crosses', 'goals_scored_from_crosses', 'xGoals_from_crosses', 'difference', 'xGoals_per_cross']
crosses_players[crosses_players.n_crosses > 50].sort_values(['xGoals_per_cross'], ascending=False)



,n_crosses,goals_scored_from_crosses,xGoals_from_crosses,difference,xGoals_per_cross
player2,,,,,
luis suarez,209,44.0,34.892571,-9.107429,0.166950
lukas podolski,61,10.0,10.158822,0.158822,0.166538
filip kostic,135,12.0,21.516195,9.516195,0.159379
moi gomez,69,14.0,10.855441,-3.144559,0.157325
aritz aduriz,92,18.0,14.387418,-3.612582,0.156385
gareth bale,150,33.0,23.383420,-9.616580,0.155889
bebe,59,8.0,9.185548,1.185548,0.155687
angel di maria,360,69.0,55.107913,-13.892087,0.153078
lionel messi,399,75.0,61.020699,-13.979301,0.152934


## 3.11 Most unlucky players when passing the ball

In [ ]:
print('Passes and Through-Balls:')
assisting_players.sort_values(['difference'], ascending=False)

Passes and Through-Balls:


,n_passes,goals_scored_from_passes,xGoals_from_passes,difference,xGoals_per_pass
player2,,,,,
joan verdu,134,7.0,15.435855,8.435855,0.115193
luca cigarini,164,6.0,13.924280,7.924280,0.084904
philippe coutinho,177,13.0,20.679299,7.679299,0.116832
xabi prieto,152,6.0,13.635634,7.635634,0.089708
alejandro gomez,137,5.0,12.352582,7.352582,0.090165
antonio di natale,136,7.0,14.133843,7.133843,0.103925
victor ibarbo,116,6.0,12.998082,6.998082,0.112052
eden hazard,338,27.0,33.592639,6.592639,0.099387
nolito,179,18.0,24.529997,6.529997,0.137039


In [ ]:
print('Crosses:')
crosses_players.sort_values(['difference'], ascending=False)

Crosses:


,n_crosses,goals_scored_from_crosses,xGoals_from_crosses,difference,xGoals_per_cross
player2,,,,,
johannes geis,232,12.0,24.591885,12.591885,0.106000
willian,230,13.0,24.713047,11.713047,0.107448
joan verdu,192,10.0,21.511608,11.511608,0.112040
pascal gross,139,5.0,15.668331,10.668331,0.112722
luca cigarini,278,18.0,27.916344,9.916344,0.100419
filip kostic,135,12.0,21.516195,9.516195,0.159379
ryad boudebouz,416,34.0,43.468857,9.468857,0.104492
ever banega,223,16.0,25.390751,9.390751,0.113860
ezequiel lavezzi,161,11.0,20.351468,9.351468,0.126407
